In [2]:
# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd
from sklearn.datasets import make_blobs
import mglearn
import sklearn
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler


# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
#style.use('ggplot') or plt.style.use('ggplot')

# Preprocesado y modelado
# ==============================================================================
from sklearn.ensemble import IsolationForest
from sklearn.pipeline import make_pipeline

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')


import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler

#Random global
seed = 12345
rng = np.random.default_rng(seed)  # can be called without a seed
rng.random()
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
df = pd.read_csv (r"C:\Users\White\Desktop\tfg\generados\RFECved.csv")
dfNon = pd.read_csv (r"C:\Users\White\Desktop\tfg\generados\scaled.csv")
df2 = pd.read_csv (r"C:\Users\White\Desktop\tfg\generados\Deoutliers2.csv")
dfR = pd.read_csv (r"C:\Users\White\Desktop\tfg\generados\Raw-ishDf.csv")

# GaussianNB as a baseline

sorted(sklearn.metrics.SCORERS.keys())

In [55]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
import sklearn.metrics as metrics

X = df.copy()
X["LabelA"]=df2["LabelA"]
y = X.pop("LabelA")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, random_state=42)

dfInterno=pd.DataFrame()
dfBest=pd.DataFrame()

#baseline parameters
p_grid = { 
    'var_smoothing': [1e-8, 1e-7, 1e-6, 1e-5, 1e-4]
}

scoring = {'accuracy', "precision", "recall",  "f1"}

outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
# Nested CV with parameter optimization
for train_index, test_index in outer_cv.split(X,y):
    X_train,X_test= X.iloc[train_index,:], X.iloc[test_index,:] 
    y_train,y_test= y.iloc[train_index], y.iloc[test_index]
    
    grid = GridSearchCV(GaussianNB(),return_train_score=True,scoring=scoring, refit='accuracy', param_grid=p_grid, n_jobs=-1, cv= StratifiedKFold(n_splits=5, shuffle=True, random_state=1), verbose=3)
    pipe= make_pipeline(MinMaxScaler(),StandardScaler(),grid)
    fittedgrid=pipe.fit(X_train, y_train)
#Saving the 25 subresults
    dfCurrent= pd.DataFrame(grid.cv_results_)
    dfInterno = pd.concat([dfInterno, dfCurrent], axis=0)
#Saving the best params and the results
    print(grid.best_params_)
    print(grid.best_score_)
    #dfBest= pd.concat([dfBest, dfCurrentBest], axis=0)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
{'var_smoothing': 1e-08}
0.9864184815803458
Fitting 5 folds for each of 5 candidates, totalling 25 fits
{'var_smoothing': 1e-08}
0.986673067244306
Fitting 5 folds for each of 5 candidates, totalling 25 fits
{'var_smoothing': 1e-08}
0.9863698248252482
Fitting 5 folds for each of 5 candidates, totalling 25 fits
{'var_smoothing': 1e-05}
0.9841536627722809
Fitting 5 folds for each of 5 candidates, totalling 25 fits
{'var_smoothing': 1e-08}
0.9847713463556648


In [20]:
dfInterno.to_csv(r"C:\Users\White\Desktop\tfg\generados\GaussianGridSearchCV.csv", index = None, header=True) 

# RandomForest

In [9]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics as metrics

X = df.copy()
X["LabelA"]=df2["LabelA"]
y = X.pop("LabelA")

dfInterno=pd.DataFrame()
# define the model with default hyperparameters
model=RandomForestClassifier(random_state=42)
scoring = {'accuracy', "precision", "recall",  "f1"}

# define the grid of values to search
p_grid = { 
    'n_estimators': [100, 200],
    #Empirical good default values [...] max_features=sqrt(n_features) for classification tasks
   # 'max_features': ["auto", "sqrt"],
    'max_depth' : [3,4,5],
    #'criterion' :['gini', 'entropy']
}

outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
# Nested CV with parameter optimization
for train_index, test_index in outer_cv.split(X,y):
    X_train,X_test= X.iloc[train_index,:], X.iloc[test_index,:] 
    y_train,y_test= y.iloc[train_index], y.iloc[test_index]
        
    grid = GridSearchCV(model,return_train_score=True,scoring=scoring, refit="accuracy",param_grid=p_grid, n_jobs=-1, cv= StratifiedKFold(n_splits=5, shuffle=True, random_state=1), verbose=4)
    pipe= make_pipeline(MinMaxScaler(),StandardScaler(),grid)
    fittedgrid=pipe.fit(X_train, y_train)
#Saving the 25 subresults
    dfCurrent= pd.DataFrame(grid.cv_results_)
    dfInterno = pd.concat([dfInterno, dfCurrent], axis=0)
#Saving the best params and the results
    print(grid.best_params_)
    print(grid.best_score_)
    #dfBest= pd.concat([dfBest, dfCurrentBest], axis=0)  

Fitting 5 folds for each of 6 candidates, totalling 30 fits
{'max_depth': 5, 'n_estimators': 200}
0.999363601040338
Fitting 5 folds for each of 6 candidates, totalling 30 fits
{'max_depth': 5, 'n_estimators': 200}
0.9994085225680325
Fitting 5 folds for each of 6 candidates, totalling 30 fits
{'max_depth': 5, 'n_estimators': 100}
0.9994010361372409
Fitting 5 folds for each of 6 candidates, totalling 30 fits
{'max_depth': 5, 'n_estimators': 200}
0.9994197534754818
Fitting 5 folds for each of 6 candidates, totalling 30 fits
{'max_depth': 5, 'n_estimators': 200}
0.9993823190792801


In [10]:
dfInterno.to_csv(r"C:\Users\White\Desktop\tfg\generados\RForestGridSearchCV.csv", index = None, header=True) 

# Adaboost

In [3]:
import pandas as pd
from sklearn.model_selection import cross_val_score
import sklearn.metrics as metrics

X = df.copy()
X["LabelA"]=df2["LabelA"]
y = X.pop("LabelA")

dfInterno=pd.DataFrame()
scoring = {'accuracy', "precision", "recall",  "f1"}

from sklearn.ensemble import AdaBoostClassifier
# define the model with default hyperparameters
model = AdaBoostClassifier()
# define the grid of values to search
p_grid = dict()
p_grid['n_estimators'] = [100,200]
p_grid['learning_rate'] = [0.1,0.001,0.0001]


outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
# Nested CV with parameter optimization
for train_index, test_index in outer_cv.split(X,y):
    X_train,X_test= X.iloc[train_index,:], X.iloc[test_index,:] 
    y_train,y_test= y.iloc[train_index], y.iloc[test_index]
        
    grid = GridSearchCV(model,return_train_score=True,scoring=scoring, refit="accuracy",param_grid=p_grid, n_jobs=-1, cv= StratifiedKFold(n_splits=5, shuffle=True, random_state=1), verbose=4)
    pipe= make_pipeline(MinMaxScaler(),StandardScaler(),grid)
    fittedgrid=pipe.fit(X_train, y_train)
#Saving the 25 subresults
    dfCurrent= pd.DataFrame(grid.cv_results_)
    dfInterno = pd.concat([dfInterno, dfCurrent], axis=0)
#Saving the best params and the results
    print(grid.best_params_)
    print(grid.best_score_)
    #dfBest= pd.concat([dfBest, dfCurrentBest], axis=0)    

Fitting 5 folds for each of 6 candidates, totalling 30 fits
{'learning_rate': 0.1, 'n_estimators': 200}
0.9988395072312443
Fitting 5 folds for each of 6 candidates, totalling 30 fits
{'learning_rate': 0.1, 'n_estimators': 200}
0.9987758670339766
Fitting 5 folds for each of 6 candidates, totalling 30 fits
{'learning_rate': 0.1, 'n_estimators': 200}
0.9988170456966257
Fitting 5 folds for each of 6 candidates, totalling 30 fits
{'learning_rate': 0.1, 'n_estimators': 200}
0.9987983276576836
Fitting 5 folds for each of 6 candidates, totalling 30 fits
{'learning_rate': 0.1, 'n_estimators': 200}
0.9988170470279579


In [4]:
dfInterno.to_csv(r"C:\Users\White\Desktop\tfg\generados\AdaboostGridSearchCV.csv", index = None, header=True) 

# LinearSVC

In [2]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVC
import sklearn.metrics as metrics

X = df.copy()
X["LabelA"]=df2["LabelA"]
y = X.pop("LabelA")
dfInterno=pd.DataFrame()

X_trian, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, random_state=42)

scoring = {'accuracy', "precision", "recall",  "f1"}

# define the model with default hyperparameters
model = LinearSVC()
# define the grid of values to search
p_grid = {"tol": [1e-3, 1e-4], "C": [1, 10, 100]}


outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
# Nested CV with parameter optimization
for train_index, test_index in outer_cv.split(X,y):
    X_train,X_test= X.iloc[train_index,:], X.iloc[test_index,:] 
    y_train,y_test= y.iloc[train_index], y.iloc[test_index]
        
    grid = GridSearchCV(model,return_train_score=True,scoring=scoring, refit="accuracy",param_grid=p_grid, n_jobs=-1, cv= StratifiedKFold(n_splits=5, shuffle=True, random_state=1), verbose=4)
    pipe= make_pipeline(MinMaxScaler(),StandardScaler(),grid)
    fittedgrid=pipe.fit(X_train, y_train)
#Saving the 25 subresults
    dfCurrent= pd.DataFrame(grid.cv_results_)
    dfInterno = pd.concat([dfInterno, dfCurrent], axis=0)
#Saving the best params and the results
    print(grid.best_params_)
    print(grid.best_score_)
    #dfBest= pd.concat([dfBest, dfCurrentBest], axis=0)      

Fitting 5 folds for each of 6 candidates, totalling 30 fits
{'C': 1, 'tol': 0.0001}
0.9988731992528914
Fitting 5 folds for each of 6 candidates, totalling 30 fits
{'C': 1, 'tol': 0.001}
0.998966786714868
Fitting 5 folds for each of 6 candidates, totalling 30 fits
{'C': 1, 'tol': 0.0001}
0.998974273916431
Fitting 5 folds for each of 6 candidates, totalling 30 fits
{'C': 1, 'tol': 0.001}
0.9989443251101795
Fitting 5 folds for each of 6 candidates, totalling 30 fits
{'C': 1, 'tol': 0.001}
0.9988769435894091


In [3]:
dfInterno.to_csv(r"C:\Users\White\Desktop\tfg\generados\LinearSVCGridSearchCV.csv", index = None, header=True) 

# Autoencoder

In [ ]:
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import numpy as np
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import Adam
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler 
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
from keras.layers import Input, Dense
from keras.models import Model, Sequential
from keras import regularizers




scaler = StandardScaler()

X = df.copy()
X["LabelA"]=df2["LabelA"]

y = X.pop("LabelA")

#Autoencoder
# Function to create model, required for KerasClassifier
def create_model(learning_rate= 0.0001):
    # Building the Input Layer
    input_layer = Input(shape =(X.shape[1], ))
    model = Sequential()

    # Building the Encoder network
    encoded = Dense(100, activation ='tanh',
                activity_regularizer = regularizers.l1(10e-5))(input_layer)
    encoded = Dense(50, activation ='tanh',
                activity_regularizer = regularizers.l1(10e-5))(encoded)
    encoded = Dense(25, activation ='tanh',
                activity_regularizer = regularizers.l1(10e-5))(encoded)
    encoded = Dense(12, activation ='tanh',
                activity_regularizer = regularizers.l1(10e-5))(encoded)
    encoded = Dense(6, activation ='relu')(encoded)
  
    # Building the Decoder network
    decoded = Dense(12, activation ='tanh')(encoded)
    decoded = Dense(25, activation ='tanh')(decoded)
    decoded = Dense(50, activation ='tanh')(decoded)
    decoded = Dense(100, activation ='tanh')(decoded)
  
    # Building the Output Layer
    output_layer = Dense(X.shape[1], activation ='relu')(decoded)
    autoencoder = Model(input_layer, output_layer)

    #optimizer=tf.keras.optimizers.Adadelta(lr=0.1, rho=0.95, epsilon=None, decay=0.0)

    # Compile model
    autoencoder.compile(optimizer ="adadelta", loss ="mse")

    return autoencoder




model = KerasClassifier(build_fn = create_model)

scoring = {'accuracy', "precision", "recall",  "f1"}
dfInterno=pd.DataFrame()

# define the grid search parameters
batch_size = [100, 200]
epochs = [10,20]

p_grid = dict(batch_size=batch_size, epochs=epochs)

outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
# Nested CV with parameter optimization
for train_index, test_index in outer_cv.split(X,y):
    X_train,X_test= X.iloc[train_index,:], X.iloc[test_index,:] 
    y_train,y_test= y.iloc[train_index], y.iloc[test_index]
    
    grid = GridSearchCV(model,return_train_score=True,scoring=scoring, refit="accuracy",param_grid=p_grid, n_jobs=-1, cv= StratifiedKFold(n_splits=5, shuffle=True, random_state=1), verbose=4)
    pipe= make_pipeline(MinMaxScaler(),StandardScaler(),grid)
    fittedgrid=pipe.fit(X_train, y_train)
#Saving the 25 subresults
    dfCurrent= pd.DataFrame(grid.cv_results_)
    dfInterno = pd.concat([dfInterno, dfCurrent], axis=0)
#Saving the best params and the results
    print(grid.best_params_)
    print(grid.best_score_)
    #dfBest= pd.concat([dfBest, dfCurrentBest], axis=0)      

In [3]:
dfInterno.to_csv(r"C:\Users\White\Desktop\tfg\generados\AutoencoderGridSearchCV.csv", index = None, header=True) 

# TensorBoard tryout

In [38]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp


HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16, 32]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.2))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)
def train_test_model(hparams):
  model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation=tf.nn.relu),
    tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax),
  ])
  model.compile(
      optimizer=hparams[HP_OPTIMIZER],
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy'],
  )

  model.fit(X_train, y_train, epochs=1) # Run with 1 epoch to speed things up for demo purposes
  _, accuracy = model.evaluate(X_test, y_test)
  return accuracy

session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
  for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
    for optimizer in HP_OPTIMIZER.domain.values:
      hparams = {
          HP_NUM_UNITS: num_units,
          HP_DROPOUT: dropout_rate,
          HP_OPTIMIZER: optimizer,
      }
      run_name = "run-%d" % session_num
      print('--- Starting trial: %s' % run_name)
      print({h.name: hparams[h] for h in hparams})
      run('logs/hparam_tuning/' + run_name, hparams)
      session_num += 1


--- Starting trial: run-0
{'num_units': 16, 'dropout': 0.1, 'optimizer': 'adam'}
2087/2087 [==============================] - 2s 703us/step - loss: 0.0019 - accuracy: 0.9994
--- Starting trial: run-1
{'num_units': 16, 'dropout': 0.1, 'optimizer': 'sgd'}
2087/2087 [==============================] - 2s 732us/step - loss: 0.0092 - accuracy: 0.9977
--- Starting trial: run-2
{'num_units': 16, 'dropout': 0.2, 'optimizer': 'adam'}
2087/2087 [==============================] - 2s 704us/step - loss: 0.0018 - accuracy: 0.9996
--- Starting trial: run-3
{'num_units': 16, 'dropout': 0.2, 'optimizer': 'sgd'}
2087/2087 [==============================] - 2s 701us/step - loss: 0.0100 - accuracy: 0.9979
--- Starting trial: run-4
{'num_units': 32, 'dropout': 0.1, 'optimizer': 'adam'}
2087/2087 [==============================] - 2s 702us/step - loss: 8.0466e-04 - accuracy: 0.9999
--- Starting trial: run-5
{'num_units': 32, 'dropout': 0.1, 'optimizer': 'sgd'}
2087/2087 [==============================] - 2s 

In [39]:
%tensorboard --logdir logs/hparam_tuning



# RNN

In [2]:
# Importing the Keras libraries and packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, LSTM, Dropout, Input
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers

X = df.copy()
X["LabelA"]=df2["LabelA"]
y = X.pop("LabelA")

dfInterno=pd.DataFrame()

hidden_nodes = int(2/3 * (X.shape[1] + 1))
#print(f"The number of hidden nodes is {hidden_nodes}.")

# Function to create model, required for KerasClassifier
# Initialising the RNN
def create_model():
    #Initializing the RNN
    model = Sequential()
    #Adding the first LSTM layer and dropout 
    model.add(LSTM(units=21, return_sequences=False, input_shape=(30, 1)))
    model.add(Dropout(0.2))
    #Adding the output layer
    model.add(Dense(units=1,activation='sigmoid'))
    #Output is two classes with only one present at a time. 
    #Softmax allows the model to interpret the outputs as probabilities

    #Compiling the model without training
    from tensorflow.keras.optimizers import SGD
    opt = SGD(lr=0.01)

    model.compile(optimizer="Adam", loss="binary_crossentropy", metrics=['acc'])
    return model

model = KerasClassifier(build_fn = create_model)



# define the grid search parameters
batch_size = [1000,5000]
epochs = [10, 20]

p_grid = dict(batch_size=batch_size, epochs=epochs)

scoring = {'accuracy', "precision", "recall",  "f1"}

outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
# Nested CV with parameter optimization
for train_index, test_index in outer_cv.split(X,y):
    X_train,X_test= X.iloc[train_index,:], X.iloc[test_index,:] 
    y_train,y_test= y.iloc[train_index], y.iloc[test_index]

    grid = GridSearchCV(model,return_train_score=True,scoring=scoring, refit="accuracy",param_grid=p_grid, n_jobs=-1, cv= StratifiedKFold(n_splits=5, shuffle=True, random_state=1), verbose=4)
    
    pipe= make_pipeline(MinMaxScaler(),StandardScaler(),grid)
    fittedgrid=pipe.fit(X_train, y_train)
#Saving the 25 subresults
    dfCurrent= pd.DataFrame(grid.cv_results_)
    dfInterno = pd.concat([dfInterno, dfCurrent], axis=0)
#Saving the best params and the results
    print(grid.best_params_)
    print(grid.best_score_)
    #dfBest= pd.concat([dfBest, dfCurrentBest], axis=0)      
    

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Epoch 1/20
268/268 [==============================] - 11s 35ms/step - loss: 0.2955 - acc: 0.8650
Epoch 2/20
268/268 [==============================] - 9s 34ms/step - loss: 0.1444 - acc: 0.9482
Epoch 3/20
268/268 [==============================] - 9s 33ms/step - loss: 0.1083 - acc: 0.9638
Epoch 4/20
268/268 [==============================] - 9s 32ms/step - loss: 0.0934 - acc: 0.9733
Epoch 5/20
268/268 [==============================] - 9s 32ms/step - loss: 0.0811 - acc: 0.9786
Epoch 6/20
268/268 [==============================] - 9s 32ms/step - loss: 0.0731 - acc: 0.9816
Epoch 7/20
268/268 [==============================] - 9s 32ms/step - loss: 0.0690 - acc: 0.9827
Epoch 8/20
268/268 [==============================] - 9s 32ms/step - loss: 0.0601 - acc: 0.9847
Epoch 9/20
268/268 [==============================] - 9s 32ms/step - loss: 0.0514 - acc: 0.9864
Epoch 10/20
268/268 [==============================] - 9s 32ms/step - loss:

In [3]:
dfInterno.to_csv(r"C:\Users\White\Desktop\tfg\generados\RNNGridSearchCV.csv", index = None, header=True) 

# CNN

In [7]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import numpy as np
from tensorflow.keras.layers import Input, Dense, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers

X = df.copy()
X["LabelA"]=df2["LabelA"]
y = X.pop("LabelA")



# Function to create model, required for KerasClassifier
# Initialising the CNN
def create_model():
    model = Sequential()
    model.add(Conv1D(filters=31, kernel_size=3, activation='relu'))
    #model.add(Conv1D(filters=31, kernel_size=3, activation='relu'))
    model.add(Dropout(0.2))
    model.add(MaxPooling1D(pool_size=3))
    model.add(Flatten())
    model.add(Dense(1, activation="sigmoid")) # binary activation output
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model



model = KerasClassifier(build_fn = create_model)


# define the grid search parameters
batch_size = [32, 64]
#optimizer= ["RMSprop", "adam"]
epochs = [10, 20]
#dropout = [0, 0.1, 0.2]
p_grid = dict(batch_size=batch_size, epochs=epochs)
scoring = {'accuracy', "precision", "recall",  "f1"}
dfInterno=pd.DataFrame()


outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
# Nested CV with parameter optimization
for train_index, test_index in outer_cv.split(X,y):

    X_train,X_test= X.iloc[train_index,:], X.iloc[test_index,:] 
    y_train,y_test= y.iloc[train_index], y.iloc[test_index]
    
    X_train=MinMaxScaler().fit_transform(X_train)
    X_train=MinMaxScaler().fit_transform(X_train)

    X_test=MinMaxScaler().fit_transform(X_test)
    X_test=MinMaxScaler().fit_transform(X_test)

    #X_train=X_train.to_numpy()
    X_train = X_train[..., np.newaxis]
    #X_test=X_test.to_numpy()
    X_test = X_test[..., np.newaxis]
    
    grid = GridSearchCV(model,return_train_score=True,scoring=scoring, refit="accuracy",param_grid=p_grid, n_jobs=-1, cv= StratifiedKFold(n_splits=5, shuffle=True, random_state=1), verbose=4)
    fittedgrid=grid.fit(X_train, y_train)
#Saving the 25 subresults
    dfCurrent= pd.DataFrame(grid.cv_results_)
    dfInterno = pd.concat([dfInterno, dfCurrent], axis=0)
#Saving the best params and the results
    print(grid.best_params_)
    print(grid.best_score_)
    #dfBest= pd.concat([dfBest, dfCurrentBest], axis=0)      
    

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Epoch 1/20
8348/8348 [==============================] - 15s 2ms/step - loss: 0.0525 - accuracy: 0.9827
Epoch 2/20
8348/8348 [==============================] - 14s 2ms/step - loss: 0.0208 - accuracy: 0.9949
Epoch 3/20
8348/8348 [==============================] - 14s 2ms/step - loss: 0.0189 - accuracy: 0.9956
Epoch 4/20
8348/8348 [==============================] - 14s 2ms/step - loss: 0.0182 - accuracy: 0.9958
Epoch 5/20
8348/8348 [==============================] - 14s 2ms/step - loss: 0.0169 - accuracy: 0.9961
Epoch 6/20
8348/8348 [==============================] - 14s 2ms/step - loss: 0.0163 - accuracy: 0.9963
Epoch 7/20
8348/8348 [==============================] - 15s 2ms/step - loss: 0.0167 - accuracy: 0.9961
Epoch 8/20
8348/8348 [==============================] - 15s 2ms/step - loss: 0.0163 - accuracy: 0.9963
Epoch 9/20
8348/8348 [==============================] - 14s 2ms/step - loss: 0.0160 - accuracy: 0.9964
Epoch 10/20
8

In [8]:
dfInterno.to_csv(r"C:\Users\White\Desktop\tfg\generados\CNNGridSearchCV.csv", index = None, header=True) 